Variable decomposition is an important step in IADOPT variable modelling, It helps to identify the classes and object properties mentioed in variables. Once we decompose it, we can model it as per IADOPT rules and validate it with SHACL.

In this notebook, Instead of going with LLM (Large Language Model), which will be resource intensive, We tested Encoder based model to perform decomposition, The encoder architecture is very efficient in contextual understing and can assist in NER, relation extraction etc. In this work, we did not have many example, so we used Zero-shot NER model, which is based on BERT architecture, but also trained with large amount of data to generalize the entity type, which can be leveraged for ZERO shot NER.

In [ ]:
from gliner import GLiNER
model = GLiNER.from_pretrained("knowledgator/gliner-bi-large-v1.0")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

C:\Users\saurav\anaconda3\envs\iadopt\lib\site-packages\gliner\model.py:794: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_file, map_location=t

In [ ]:
classes =  ["Variable","Property","Entity","Constraint"]
objectProperties = ["hasProperty","hasObjectOfInterest","hasContextObject","hasMatrix","hasConstraint"]

In [29]:
import pandas as pd

In [30]:
data = pd.read_csv("I-ADOPT Variable Challenge List - Tabellenblatt1.csv")

### Automated concept identification

In [34]:
text_data = data["Variable description"].to_list()

In [58]:
entites_list = []
for idx, text in enumerate(text_data):
    entities = model.predict_entities(text, classes, threshold=0.3)
    entites_list.append((entities,idx+1))

In [61]:
flattened_list = [{"doc": sublist[1], **item} for sublist in entites_list for item in sublist[0]]

In [62]:
pd.DataFrame(flattened_list)

,doc,start,end,text,label,score
0,1,0,7,Density,Property,0.571467
1,1,56,66,Solar Wind,Entity,0.436654
2,2,0,22,Temperature of the air,Property,0.362974
3,2,28,34,height,Property,0.437001
4,3,14,24,cloudiness,Property,0.385806
...,...,...,...,...,...,...
150,27,0,18,Mass concentration,Property,0.410865
151,27,22,33,isobutylene,Varaible,0.345103
152,27,37,47,chloroform,Varaible,0.331018
153,28,22,34,3-star hotel,Property,0.364573
